In [51]:
!pip install yfinance pandas_datareader plotly matplotlib

     ---------------------------------------- 7.7/7.7 MB 25.7 MB/s eta 0:00:00
     ------------------------------------- 187.5/187.5 kB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB ? eta 0:00:00
     ---------------------------------------- 103.2/103.2 kB ? eta 0:00:00
  Using cached pillow-10.3.0-cp310-cp310-win_amd64.whl (2.5 MB)
     ---------------------------------------- 2.2/2.2 MB 46.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1) Question 1: [Macro] Average growth of GDP in 2023

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [23]:
GDPC1 = pd.read_csv('data\GDPC1.csv')

In [24]:
GDPC1['gdp_us_yoy'] = (GDPC1.GDPC1/GDPC1.GDPC1.shift(4)-1)*100 #Part 1
GDPC1['gdp_yoy_mean'] = round(GDPC1['gdp_us_yoy'].rolling(4).mean(), 1) # part 2 = 2.5

GDPC1.tail()


,DATE,GDPC1,gdp_us_yoy,gdp_yoy_mean
303,2022-10-01,21989.981,0.651692,2.0
304,2023-01-01,22112.329,1.717927,1.5
305,2023-04-01,22225.350,2.382468,1.6
306,2023-07-01,22490.692,2.926887,1.9
307,2023-10-01,22679.255,3.134491,2.5


# Question 2. [Macro] Inverse "Treasury Yield"

In [38]:
DGS2 = pd.read_csv('data\DGS2.csv')
DGS10 = pd.read_csv('data\DGS10.csv')

In [49]:
merged_df = pd.merge(DGS2, DGS10, on='DATE')
# Convert columns to numeric
merged_df['DGS2'] = pd.to_numeric(merged_df['DGS2'], errors='coerce')
merged_df['DGS10'] = pd.to_numeric(merged_df['DGS10'], errors='coerce')

# Filter out rows that are not formatted as float
merged_df = merged_df[(merged_df['DGS2'].notnull()) & (merged_df['DGS10'].notnull())]
# Reset the index
merged_df.reset_index(drop=True, inplace=True)
merged_df['DGS_DIFF'] = (merged_df.DGS10-merged_df.DGS2)
lowest_dgs_diff = round(merged_df['DGS_DIFF'].min(),1)
print(lowest_dgs_diff) # -1.1
merged_df.tail()

# merged_df.to_csv("TEST.csv")

-1.1


,DATE,DGS2,DGS10,DGS_DIFF
6073,2024-04-12,4.88,4.50,-0.38
6074,2024-04-15,4.93,4.63,-0.30
6075,2024-04-16,4.97,4.67,-0.30
6076,2024-04-17,4.93,4.59,-0.34
6077,2024-04-18,4.98,4.64,-0.34


# Question 3. [Index] Which Index is better recently?

In [71]:
# INDEXES from Yahoo Finance
# WEB: https://finance.yahoo.com/quote/%5EMXX/
SP500 = yf.download(tickers = "^MXX",
                     period = "max",
                     interval = "1d")
# WEB: https://finance.yahoo.com/quote/%5EGSPC/
IPCMEXICO = yf.download(tickers = "^GSPC",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [75]:
# Calculate the percentage growth for SP500
# print(SP500)
sp500_growth = ((SP500['Close'].loc['2024-04-09'] - SP500['Close'].loc['2019-04-09']) / SP500['Close'].loc['2019-04-09']) * 100

# Calculate the percentage growth for IPCMEXICO
# print(IPCMEXICO['Adj Close'].loc['2024-04-09'])
ipcmexico_growth = ((IPCMEXICO['Adj Close'].loc['2024-04-09'] - IPCMEXICO['Adj Close'].loc['2019-04-09']) / IPCMEXICO['Adj Close'].loc['2019-04-09']) * 100

# Compare the growth and select the higher growing index
if sp500_growth > ipcmexico_growth:
    higher_growing_index = 'SP500'
    growth_percentage = round(sp500_growth,1 )
else:
    higher_growing_index = 'IPCMEXICO'
    growth_percentage = round(ipcmexico_growth,1)

higher_growing_index, growth_percentage

('IPCMEXICO', 81.0)

# Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

In [77]:
SR = yf.download(tickers = "2222.SR",
                     period = "max",
                     interval = "1d")
BRKB = yf.download(tickers = "BRK-B",
                     period = "max",
                     interval = "1d")
AAPL = yf.download(tickers = "AAPL",
                     period = "max",
                     interval = "1d")
MSFT = yf.download(tickers = "MSFT",
                     period = "max",
                     interval = "1d")
GOOG = yf.download(tickers = "GOOG",
                     period = "max",
                     interval = "1d")
JPM = yf.download(tickers = "JPM",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [86]:
# Filter AAPL DataFrame
AAPL = AAPL.loc[AAPL.index.year == 2023]
AAPL_largest =(AAPL['Adj Close'].max()-AAPL['Adj Close'].min())/AAPL['Adj Close'].max()
print("AAPL_largest", AAPL_largest)

# Filter BRKB DataFrame
BRKB = BRKB.loc[BRKB.index.year == 2023]
BRKB_largest =(BRKB['Adj Close'].max()-BRKB['Adj Close'].min())/BRKB['Adj Close'].max()
print("BRKB_largest", BRKB_largest)

# Filter GOOG DataFrame
GOOG = GOOG.loc[GOOG.index.year == 2023]
GOOG_largest =(GOOG['Adj Close'].max()-GOOG['Adj Close'].min())/GOOG['Adj Close'].max()
print("GOOG_largest", GOOG_largest)

# Filter SR DataFrame
SR = SR.loc[SR.index.year == 2023]
SR_largest =(SR['Adj Close'].max()-SR['Adj Close'].min())/SR['Adj Close'].max()
print("SR_largest", SR_largest)

# Filter JPM DataFrame
JPM = JPM.loc[JPM.index.year == 2023]
JPM_largest =(JPM['Adj Close'].max()-JPM['Adj Close'].min())/JPM['Adj Close'].max()
print("JPM_largest", JPM_largest)

# Filter MSFT DataFrame
MSFT = MSFT.loc[MSFT.index.year == 2023]
MSFT_largest =(MSFT['Adj Close'].max()-MSFT['Adj Close'].min())/MSFT['Adj Close'].max()
print("MSFT_largest", MSFT_largest) # 0.42 Largest



AAPL_largest 0.37244419224463476
BRKB_largest 0.20775750091289963
GOOG_largest 0.3924520921912013
SR_largest 0.21393070882746015
JPM_largest 0.28249929679343244
MSFT_largest 0.4242066914981641


# Question 5. [Stocks] Dividend Yield

In [104]:
SRDivs = yf.Ticker("2222.SR")
SRDivs = SRDivs.get_actions()
BRKBDivs = yf.Ticker("BRK-B")
BRKBDivs = BRKBDivs.get_actions()
AAPLDivs = yf.Ticker("AAPL")
AAPLDivs = AAPLDivs.get_actions()
MSFTDivs = yf.Ticker("MSFT")
MSFTDivs = MSFTDivs.get_actions()
GOOGDivs = yf.Ticker("GOOG")
GOOGDivs = GOOGDivs.get_actions()
JPMDivs = yf.Ticker("JPM")
JPMDivs = JPMDivs.get_actions()

In [107]:
# Filter SR DataFrame
SRDivs = SRDivs.loc[SRDivs.index.year == 2023]
SR_Divs_paid = SRDivs['Dividends'].sum()
SR_last_OHLC = SR.iloc[-1]
SR_largest_yield = SR_Divs_paid / SR_last_OHLC['Adj Close'] * 100
print("SR_yield", SR_largest_yield)

# Filter BRKB DataFrame
BRKBDivs = BRKBDivs.loc[BRKBDivs.index.year == 2023]
BRKB_Divs_paid = SRDivs['Dividends'].sum()
BRKB_last_OHLC = BRKB.iloc[-1]
BRKB_largest_yield = BRKB_Divs_paid / BRKB_last_OHLC['Adj Close'] * 100
print("BRKB_yield", BRKB_largest_yield)

# Filter GOOG DataFrame
GOOGDivs = GOOGDivs.loc[GOOGDivs.index.year == 2023]
GOOG_Divs_paid = GOOGDivs['Dividends'].sum()
GOOG_last_OHLC = GOOG.iloc[-1]
GOOG_largest_yield = GOOG_Divs_paid / GOOG_last_OHLC['Adj Close'] * 100
print("GOOG_yield", GOOG_largest_yield) 

# Filter AAPL DataFrame
AAPLDivs = AAPLDivs.loc[AAPLDivs.index.year == 2023]
AAPL_Divs_paid = AAPLDivs['Dividends'].sum()
AAPL_last_OHLC = AAPL.iloc[-1]
AAPL_largest_yield = AAPL_Divs_paid / AAPL_last_OHLC['Adj Close'] * 100
print("AAPL_yield", AAPL_largest_yield)

# Filter JPM DataFrame
JPMDivs = JPMDivs.loc[JPMDivs.index.year == 2023]
JPM_Divs_paid = JPMDivs['Dividends'].sum()
JPM_last_OHLC = JPM.iloc[-1]
JPM_largest_yield = JPM_Divs_paid / JPM_last_OHLC['Adj Close'] * 100
print("JPM_yield", JPM_largest_yield)

# Filter JPM DataFrame
MSFTDivs = MSFTDivs.loc[MSFTDivs.index.year == 2023]
MSFT_Divs_paid = MSFTDivs['Dividends'].sum()
MSFT_last_OHLC = MSFT.iloc[-1]
MSFT_largest_yield = MSFT_Divs_paid / MSFT_last_OHLC['Adj Close'] 
print("MSFT_yield", MSFT_largest_yield)

SR_yield 2.7743470474138237
BRKB_yield 0.2553591629699064
GOOG_yield 0.0
AAPL_yield 0.4940592304162832
JPM_yield 2.4096079718909573
MSFT_yield 0.7433143940964608


# Question 6. [Exploratory] Investigate new metrics

Alpha:
Would help assess whether a strategy has outperformed or underperformed relative to a benchmark, taking into account the strategy's risk.

Sharpe Ratio:
Would help compare the returns of different strategies while considering the level of risk taken.


# Question 7. [Exploratory] Time-driven strategy description around earnings releases

What is "future events data"?

Would do a analysis on the following elements on a set of stocks to benchmark them:
- How close is the next earnings release / how long was the last earnings release.
- Average stock price movement before/after earnings.
- Volatility before/after earnings.
- Historical earnings surprise (actual earnings vs. predicted estimates).

Would have to investigate how to calculate volatility.